In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext training_rl
%set_random_seed 12

In [ ]:
%presentation_style

In [ ]:
%load_latex_macros

In [ ]:
%autoreload
import warnings

import casadi
import control as ct
import do_mpc
import gymnasium as gym
import matplotlib.pyplot as plt
import matplotx
import mediapy as media
import mujoco
import numpy as np
import seaborn as sns
from IPython.display import HTML
from ipywidgets import interact, widgets
from matplotlib.animation import FuncAnimation
from numpy.typing import NDArray
from scipy.signal import find_peaks


from training_rl.environment import (
    create_inverted_pendulum_environment,
    create_mass_spring_damper_environment,
)
from training_rl.control import (
    create_shortest_path_graph,
    plot_shortest_path_graph,
    plot_all_paths_graph,
)

warnings.simplefilter("ignore", UserWarning)
sns.set_theme()
plt.rcParams["figure.figsize"] = [9, 5]

<img src="_static/images/aai-institute-cover.svg" alt="presentation first slide" style="width:100%;">
<div class="md-slide title">Control and Planning</div>

# Introduction

In previous sections, we have designed feedback controllers for various systems with the goal of regulating the system output to a desired setpoint. Specifically, we utilized Fullstate Feedback and PID controllers. While these simple controllers can effectively regulate many systems, they have limitations that prevent high performance control for more complex systems.

First, these controllers have a fixed, static gain that does not change over time. This limits their ability to adapt to changing system dynamics or disturbances. Second, after developing a mathematical model of the system dynamics, we did not actually utilize the model in the controller design. A model-based approach could allow us to leverage our understanding of the system to improve control performance.

One ubiquitous challenge in control system design is the presence of control input constraints. Actuators in physical systems inherently have limits on their amplitude and rate of change. For example, a motor has maximum torque and acceleration limits. If control input constraints are ignored in the controller design, the resulting control inputs may saturate the actuators, degrading closed-loop performance.

There are two main strategies to address control input constraints:

- Reduce the performance requirements to levels achievable with a linear controller without violating the constraints.
- Directly account for the constraints by modifying the control design and online optimization of the control input.   For example, model predictive control utilizes the system model to predict future behavior and optimize the control input while satisfying constraints.

In subsequent sections, we will explore model-based and optimization-based control techniques to overcome the limitations of basic controllers. Properly managing control input constraints is crucial to achieving high performance in real control systems.

# Systems

## Mass-Spring-Damper Model


<div>
<figure style="float: left; width: 50%;">
    <img src="_static/images/20_mass_spring_damper.svg" width="50%"/>
    <figcaption>
        Classic model used for deriving the equations of a mass spring damper model <a href="#wiki_mass_spring_damper"><b id="wiki_mass_spring_damper-back">[Wiki Mass-Spring-Damper, 2023]</b></a>
    </figcaption>
</figure>
<div style="float: left; width: 40%;">
    
- $z(t)$: Distance along the vertical axis from some reference point.
- $m$: Mass of the object.
- $\lambda$: Coefficient of elasticity.
- $l$: Length of spring.
- $k = \frac{\lambda}{l}$
- $c$: Damping coefficient.
- $f(t)$: Force applied on the object.
- $g$: Gravity.
</div>
</div>

The system has linear state-space model with matrices:

$$
A
=
\begin{bmatrix}
0 & 1 \\
-\frac{k}{m} & -\frac{c}{m}\\
\end{bmatrix};
B
=
\begin{bmatrix}
0 \\
\frac{1}{m}\\
\end{bmatrix};
C = \begin{bmatrix}
1 & 0 \\
\end{bmatrix};
D = \begin{bmatrix}
0
\end{bmatrix}
$$

In [ ]:
m = 0.51
c = 1.52
k = 50.15
# Dynamics matrix
A = np.array(
    [
        [0, 1],
        [-k / m, -c / m],
    ]
)
# Input matrix
B = np.array([[0, 1 / m]]).transpose()
# Output matrices
C = np.array(
    [
        [1, 0],
    ]
)
D = np.zeros(1)

In [ ]:
mass_spring_damper = do_mpc.model.LinearModel("continuous")
mass_spring_damper.set_variable(var_type="_x", var_name="position")
mass_spring_damper.set_variable(var_type="_x", var_name="velocity")
mass_spring_damper.set_variable(var_type="_u", var_name="force")
mass_spring_damper.setup(A, B, C, D)

In [ ]:
env = create_mass_spring_damper_environment()
mass_spring_damper = mass_spring_damper.discretize(env.dt)

## Inverted Pendulum

<div>
<figure style="float: left; width: 40%;">
    <img src="_static/images/20_inverted_pendulum.svg" width="50%"/>
    <figcaption>
        Inverted pendulum model <a href="#goodwin_control_2000"><b id="goodwin_control_2000-back">[Goodwin et al., 2000]</b></a>
    </figcaption>
</figure>
<div style="float: left; width: 50%">
    
- $y(t)$: distance along the horizontal axis from some reference point.
- $\theta(t)$: angle of the pendulum.
- $M$: mass of the cart.
- $m$: mass of the pendulum (assumed to be concentrated at the tip).
- $l$: length of the pendulum.
- $f(t)$: force applied on the cart.
</div>
</div>

The system has the following full non-linear state space model:

$$
\dot{X}(t) = \begin{bmatrix}
\dot{x_1}(t) \\ \dot{x_2}(t) \\ \dot{x_3}(t) \\ \dot{x_4}(t)
\end{bmatrix} =
\begin{bmatrix}
x_2(t) \\
x_4(t) \\
\frac{1}{\lambda_m + \sin^2 x_2(t)} \left[
\frac{u(t)}{m} + x_4(t)^2 l \sin x_2(t) - g \cos x_2(t) \sin x_2(t)
\right] \\
\frac{1}{l\lambda_m + \sin^2 x_2(t)} \left[
-\frac{u(t)}{m}\cos x_2(t) + x_4(t)^2 l \sin x_2(t) \cos x_2(t) + (1 - \lambda_m) \sin x_2(t)
\right] \\
\end{bmatrix}
$$

And the following partial (pendulum angle only) non-linear state space model:

$$
\dot{X}(t) = \begin{bmatrix}
\dot{x_1}(t) \\ \dot{x_2}(t)
\end{bmatrix} =
\begin{bmatrix}
x_2(t) \\
\frac{1}{l\lambda_m + \sin^2 x_1(t)} \left[
-\frac{u(t)}{m}\cos x_1(t) + x_2(t)^2 l \sin x_1(t) \cos x_1(t) + (1 - \lambda_m) \sin x_1(t)
\right] \\
\end{bmatrix}
$$

And the following linearized (around $\theta = 0$ and $\dot{\theta} = 0$) state space model:

$$
A = \begin{bmatrix}
0 & 1 \\
\frac{(M + m)g}{Ml} & 0\\
\end{bmatrix};
B = \begin{bmatrix}
0 \\ -\frac{1}{Ml}
\end{bmatrix};
C = \begin{bmatrix}
1 & 0 \\
\end{bmatrix};
D = \begin{bmatrix}
0
\end{bmatrix}
$$

In [ ]:
env = create_inverted_pendulum_environment()
g = 9.81
l = env.model.geom_pos[2, 2]
m = env.model.body_mass[2]
M = env.model.body_mass[1]
lambda_m = M / m

# Dynamics matrix
A = np.array(
    [
        [0, 1],
        [(1 + lambda_m) * g / (lambda_m * l), 0],
    ]
)
# Input matrix
B = np.array([[0, -1 / (M * l)]]).transpose()
# Output matrices
C = np.array(
    [
        [1, 0],
    ]
)
D = np.zeros(1)

In [ ]:
inverted_pendulum_lin = do_mpc.model.LinearModel("continuous")
theta = inverted_pendulum_lin.set_variable(var_type="_x", var_name="theta")
dtheta = inverted_pendulum_lin.set_variable(var_type="_x", var_name="dtheta")
inverted_pendulum_lin.set_variable(var_type="_u", var_name="force");

We also define the pendulum's kinetic and potential energies:

$E_{\text{kinetic}} = \frac{1}{2} m \left( (l x_2 \cos(x_1))^{2} + (l x_2 \sin(x_1))^{2} \right)$
    
$E_{\text{potential}} = m g  l \cos(x_1)$

In [ ]:
# Energies
E_kin = 0.5 * m * ((l * dtheta * casadi.cos(theta)) ** 2 + (l * dtheta * casadi.sin(theta)) ** 2)
E_pot = m * g * l * casadi.cos(theta)
inverted_pendulum_lin.set_expression("E_kinetic", E_kin)
inverted_pendulum_lin.set_expression("E_potential", E_pot);

In [ ]:
inverted_pendulum_lin.setup(A, B, C, D)

In [ ]:
env = create_inverted_pendulum_environment()
inverted_pendulum_lin = inverted_pendulum_lin.discretize(env.dt)

We also define the full non-linear model

In [ ]:
inverted_pendulum = do_mpc.model.Model("continuous")

position = inverted_pendulum.set_variable(var_type="_x", var_name="position")
theta = inverted_pendulum.set_variable(var_type="_x", var_name="theta")
velocity = inverted_pendulum.set_variable(var_type="_x", var_name="velocity")
dtheta = inverted_pendulum.set_variable(var_type="_x", var_name="dtheta")
u = inverted_pendulum.set_variable(var_type="_u", var_name="force")

denominator = lambda_m + casadi.sin(theta) ** 2
nominator = (
    u / m * casadi.cos(theta)
    + dtheta**2 * l * casadi.sin(theta)
    - g * casadi.sin(theta) * casadi.cos(theta)
)

inverted_pendulum.set_rhs("position", velocity)
inverted_pendulum.set_rhs("velocity", nominator / denominator)

denominator = l * lambda_m + casadi.sin(theta) ** 2
nominator = (
    -u / m * casadi.cos(theta)
    + dtheta**2 * l * casadi.sin(theta) * casadi.cos(theta)
    + (1 - lambda_m) * casadi.sin(theta)
)

inverted_pendulum.set_rhs("theta", dtheta)
inverted_pendulum.set_rhs("dtheta", nominator / denominator)

And kinetic and potential energies:

$E_{\text{kinetic}} = E_{\text{kinetic, cart}} + E_{\text{kinetic, pendulum}}$
    
$E_{\text{kinetic, cart}} = \frac{1}{2} M x_3^{2}$
    
$E_{\text{kinetic, pendulum}} = \frac{1}{2} m \left( (x_3 + l x_4 \cos(x_2))^{2} + (l x_4 \sin(x_2))^{2} \right)$
    
$E_{\text{potential}} = m g  l \cos(x_2)$

In [ ]:
# Energies
E_kin = 0.5 * M * velocity**2 + 0.5 * m * (
    (velocity + l * dtheta * casadi.cos(theta)) ** 2 + (l * dtheta * casadi.sin(theta)) ** 2
)
E_pot = m * g * l * casadi.cos(theta)
inverted_pendulum.set_expression("E_kinetic", E_kin)
inverted_pendulum.set_expression("E_potential", E_pot);

In [ ]:
inverted_pendulum.setup()

# Optimal Control

Optimal control theory is a branch of control theory that deals with finding a control for a dynamical system over a period of time such that an objective function is optimized. The fundamental idea in optimal control is to formulate the goal of control as the long-term optimization of a scalar cost function.

The optimal control problem is to find a control $u^* \in \mathbf{U}$ which causes the system $\dot{x}(t) = f(x(t), u(t))$ to follow a trajectory $x^* \in \mathbf{X}$ that minimizes the cost (performance measure):


### Continuous-time

$$
\begin{array}\\
\displaystyle  \min_{x, u} & J(x, u) & \text{(cost)}\\
\text{subject to} & \dot{x}(t) = f(x(t), u(t)) & \text{(dynamical feasibility)}\\
& x(t_0) = x_0, x(T) \in \mathbf{X_T} & \text{(boundary conditions)}\\
& x(t) \in \mathbf{X} , \forall t \in [0, T] & \text{(state constraints)}\\
& u(t) \in \mathbf{U}, \forall t \in [0, T] & \text{(input constraints)}\\
\end{array}
$$

### Discrete-time

$$
\begin{array}\\
\displaystyle  \min_{x, u} & J(x, u) & \text{(cost)}\\
\text{subject to} & x_{t+1} = f(x_t, u_t) & \text{(dynamical feasibility)}\\
& x_{t_0} = x_0, x_N \in \mathbf{X_N} & \text{(boundary conditions)}\\
& x_t \in \mathbf{X} , \forall t \in \{0, 1, \dots , N - 1\} & \text{(state constraints)}\\
& u_t \in \mathbf{U}, \forall t \in \{0, 1, \dots , N - 1\} & \text{(input constraints)}\\
\end{array}
$$

### Finite Horizon 

- Continuous-time:

$$
J_{T}(x_0, u) = l_f(x(T)) + \int \limits_{0}^{T} l(x(t), u(t)) dt
$$

- Discrete-time:

$$
J_N(x_0, u) = l_f(x_N) + \sum \limits_{k = 0}^{N-1} l(x_k, u_k)
$$

### Infinite Horizon

- Continuous-time:

$$
J(x_0, u) = \int \limits_{0}^{\infty} l(x(t), u(t)) dt
$$

- Discrete-time:

$$
J(x_0, u) = \sum \limits_{k = 0}^{\infty} l(x_k, u_k)
$$

This approach is powerful for a number of reasons. First and foremost, it is very general - allowing us to specify the goal of control equally well for fully- or under-actuated, linear or nonlinear, deterministic or stochastic, and continuous or discrete systems. Second, it permits concise descriptions of potentially very complex desired behaviours, specifying the goal of control as a scalar objective (plus a list of constraints).

## Choosing the cost function

Choosing a cost function means translating the system's desired physical state into a mathematical formulation.

Examples:

- Minimum-time problems: $J = t_f - t_0$
- Terminal control problems: $J = || x(t_f) - r(t_f) ||^2$
- Minimum control-effort problems: $J = \sum \limits_{k = t_0}^{t_f} |u(t_k)|$

## Questions

- What are possible cost functions for the 2 systems?

## Dynamic Programming

Dynamic programming (DP) is a method that in general solves optimization problems that involve making a sequence of decisions by determining, for each decision, subproblems that can be solved in like fashion, such that an optimal
solution of the original problem can be found from optimal solutions of sub-
problems. This method is based on Bellman’s Principle of Optimality, which
he phrased as follows:

> An optimal policy has the property that whatever the initial state and
initial decision are, the remaining decisions must constitute an optimal
policy with regard to the state resulting from the first decision.

Dynamic Programming is a very general solution method for problems which have two properties:

- Optimal substructure (Principle of optimality applies)
  - Optimal solution can be decomposed into subproblems, e.g., shortest path
- Overlapping subproblems
  - Subproblems recur many times
  - Solutions can be cached and reused

Dynamic programming is used across a wide variety of domains, e.g.

- Scheduling algorithms
- Graph algorithms (e.g., shortest path algorithms)
- Graphical models in ML (e.g., Viterbi algorithm)
- Bioinformatics (e.g., Sequence alignment, Protein folding) 

We define the value functions as (for the sake of simplicity we will focus on the discrete-time case):

$$
V_N(x_0) := \min_{u \in \mathbf{U}} J_N(x_0, u)
$$

An admissible control sequence $u^*$ is called optimal, if

$$
V_N(x_0) = J_N(x_0, u^*)
$$

For any feasible $x_0 \in \mathbf{X}$ the optimal value function satisfies

$$
V_N(x_0) = \displaystyle \min_{u \in \mathbf{U} \\ \displaystyle f(x_0, u) \in \mathbf{X}} l(x_0, u) + V_{N-1}(f(x_0, u))
$$

Moreover, if $u^*$ is an optimal control, then

$$
V_N(x_0) = l(x_0, u^*(0)) + V_{N-1}(f(x_0, u^*(0)))
$$

###  DP Algorithm

For every initial state $x_0$, the optimal cost is equal to $V_N(x_0)$, given by the last step of the following algorithm, which proceeds backward in time from stage $N-1$ to stage $0$:

$$
V_N(x_N) = l(x_N)\\
V_k = \displaystyle \min_{u \in \mathbf{U}} \left\{ l(x_k, u_k) + V_{k+1}(f(x_k, u_k)) \right\}
$$

The optimal cost is then equal to:

$$
J^*(x_0) = V_0(x_0)
$$

## Exercise

In [ ]:
G = create_shortest_path_graph()
plot_shortest_path_graph(G)

We wish to travel from node A to node G at minimum cost. If the cost represents time then we want to find the shortest path from A to G.

- Arrows (edges) indicate the possible movements.
- Numbers on edges indicate the cost of moving along an edge.

Use Dynamic Programming to solve this problem.

> **Hint** Determine all possible paths first and then compute the cost-to-go at each node.

## Solution

In [ ]:
plot_all_paths_graph(G)

Each node in this new graph represents a state. We will start from the tail (the last states) and compute recursively the cost for each state transition.

We start by determining the cost-to-go for all positions.

Let $l(n_1, n_2)$ the cost of going from node $n_1$ to $n_2$ and $V(n)$ be the cost-to-go from node $n$.

$$
\begin{array}\\
V(\text{ABDF}) &= l(\text{ABDF}, \text{ABDFG}) &= 1\\
V(\text{ABE}) &= l(\text{ABE}, \text{ABEG}) &= 4\\
V(\text{ACF}) &= l(\text{ACF}, \text{ACFG}) &= 1\\
V(\text{ADF}) &= l(\text{ADF}, \text{ADFG}) &= 1\\
\end{array}
$$

$$
\begin{array}\\
V(\text{ABD}) &= \min \left[ l(\text{ABD}, \text{ABDG}), l(\text{ABD}, \text{ABDF}) + V(\text{ABDF}) \right]
&= \min \left[ 8, 5 + 1 \right] &= 6
\\
V(\text{AB}) &= \min \left[ l(\text{AB}, \text{ABD}) + V(\text{ABD}), l(\text{AB}, \text{ABE}) + V(\text{ABE}) \right]
&= \min \left[ 9 + 5, 6 + 4 \right] &= 10
\\
V(\text{AC}) &= l(\text{AC}, \text{ACF}) + V(\text{ACF}) &= 2 + 1 &= 3
\\
V(\text{AD}) &= \min \left[ l(\text{AD}, \text{ADF}) + V(\text{ADF}), l(\text{AD}, \text{ADG})) \right]
&= \min \left[ 5 + 1, 8 \right] &= 6
\\
\end{array}
$$

$$
\begin{array}\\
V(\text{A}) &= \min \left[
l(\text{A}, \text{AB}) + V(\text{AB}), l(\text{A}, \text{AC}) + V(\text{AC}), l(\text{A}, \text{AD}) + V(\text{AD})
\right]
&= \min \left[ 1 + 10, 5 + 3, 3 + 6 \right] &= 8
\\
\end{array}
$$

The shortest-path is aCFG.

In [ ]:
plot_all_paths_graph(G, show_solution=True)

## Linear Quadratic Regulator

While solving the optimal control problem (OCP) is very hard in general, there are a few very important special cases where the solutions are very accessible. Most of these involve variants on the case of linear dynamics and quadratic cost. The simplest case, called the linear quadratic regulator (LQR), is formulated as stabilizing a time-invariant linear system to the origin can be solved analytically.

For a discrete-time linear system described by:

$$
\mathbf{x}_{t+1} = A \mathbf{x}_t + B \mathbf{u}_t
$$

where $x\in \mathbb {R} ^{n}$ (that is, $x$ is an $n$-dimensional real-valued vector) is the state of the system and $u\in \mathbb {R} ^{m}$ is the control input. Given a quadratic cost function for the system, defined as:

### Finite-Horizon

$$
J_0(\mathbf{x}_0, \mathbf{u}) = \frac{1}{2} \mathbf{x}_N^T Q \mathbf{x}_N + \frac{1}{2} \sum \limits _{k = 0}^{N - 1}  \mathbf{x}_k^{T}Q \mathbf{x}_k + \mathbf{u}_k^{T} R \mathbf{u}_k
$$

With $Q = Q^T \succeq 0$, $R = R^T \succeq 0$.

### Infinite-Horizon

$$
J(\mathbf{x}_0, \mathbf{u}) = \frac{1}{2} \sum \limits _{k = 0}^{\infty} \mathbf{x}_k^{T}Q \mathbf{x}_k + \mathbf{u}_k^{T} R \mathbf{u}_k
$$

With $Q = Q^T \succeq 0$, $R = R^T \succeq 0$.

Let's solve this for the finite-horizon case using dynamic programming. We start by setting:

$$
V_N(\mathbf{x}_N) = J_N(\mathbf{x}_N) = \frac{1}{2} \mathbf{x}_{N}^T Q \mathbf{x}_{N} = \frac{1}{2} \mathbf{x}_{N}^T P \mathbf{x}_{N}
$$

And then proceed backward in time:

$$
\begin{array}\\
V_{N-1}(\mathbf{x}_{N-1}) &=& \displaystyle \min_{\mathbf{u}_{N-1}} J_{N-1}(\mathbf{x}_{N-1}, \mathbf{u}_{N-1})\\
&=& \displaystyle \min_{\mathbf{u}_{N-1}} \frac{1}{2} \left(
\mathbf{x}_{N-1}^{T} Q \mathbf{x}_{N-1} + \mathbf{u}_{N-1}^{T} R \mathbf{u}_{N-1} + \mathbf{x}_{N}^T P \mathbf{x}_{N}
\right) \\
&=& \displaystyle \min_{\mathbf{u}_{N-1}} \frac{1}{2} \left(
\mathbf{x}_{N-1}^{T} Q \mathbf{x}_{N-1} + \mathbf{u}_{N-1}^{T} R \mathbf{u}_{N-1} + (A\mathbf{x}_{N-1} + B\mathbf{u}_{N-1})^T P (A\mathbf{x}_{N-1} + B\mathbf{u}_{N-1})
\right)
\end{array}
$$

Taking the gradient with respect to $\mathbf{u}_{N-1}$:

$$
\displaystyle \nabla_{\mathbf{u}_{N-1}} J_{N-1}(\mathbf{x}_{N-1}, \mathbf{u}_{N-1}) = 
R \mathbf{u}_{N-1} + B^T P (A \mathbf{x}_{N-1} + B \mathbf{u}_{N - 1}) = 0
$$

Gives us the following optimal feedback control at step $N - 1$:

$$
\mathbf{u}^*_{N-1} = -(R + B^T P B)^{-1} B^T P B \mathbf{x}_{N-1} = - K \mathbf{x}_{N-1}
$$

With $K = (R + B^T P B)^{-1} B^T P B$

The optimal cost-to-go is then:

$$
\begin{array}\\
V_{N-1}(\mathbf{x}_{N-1}) &= J_{N-1}(\mathbf{x}_{N-1}, \mathbf{u}^*_{N-1}) \\
&= \frac{1}{2}  \left(
\mathbf{x}_{N-1}^{T} Q \mathbf{x}_{N-1} + \mathbf{u}_{N-1}^{*T} R \mathbf{u}^{*}_{N-1} + \mathbf{x}_{N}^T P \mathbf{x}_{N}
\right)\\
&= \frac{1}{2}  \left(
\mathbf{x}_{N-1}^{T} Q \mathbf{x}_{N-1} + \mathbf{u}_{N-1}^{*T} R \mathbf{u}^{*}_{N-1} + (A\mathbf{x}_{N-1} + B\mathbf{u}_{N-1})^T P (A\mathbf{x}_{N-1} + B\mathbf{u}_{N-1})
\right)\\
&= \frac{1}{2}  \left(
\mathbf{x}_{N-1}^{T} Q \mathbf{x}_{N-1} + \mathbf{x}_{N-1}^{T} K^T R K \mathbf{x}_{N-1} + \mathbf{x}_{N-1}^T(A - BK)^T P (A - BK)\mathbf{x}_{N-1}
\right)\\
&= \frac{1}{2}  
\mathbf{x}_{N-1}^{T} \left(
Q + A^{T}PA-(A^{T}PB)(R+B^{T}PB)^{-1}(B^{T}PA)
\right) \mathbf{x}_{N-1}
\\
&:= \frac{1}{2} \mathbf{x}_{N-1}^T P \mathbf{x}_{N-1}\\
\end{array}
$$

The last step is needed to ensure that the derivation works recursively for all steps.

and $P$ is found by solving the discrete time algebraic Riccati equation (DARE):

$$
Q + A^{T}PA-(A^{T}PB)(R+B^{T}PB)^{-1}(B^{T}PA) = P.
$$

### Mass-Spring-Damper

In [ ]:
# Initialize the controller
lqr = do_mpc.controller.LQR(mass_spring_damper)

# Initialize the parameters
env = create_mass_spring_damper_environment()
lqr.settings.t_step = env.dt
lqr.settings.n_horizon = None  # infinite horizon

# Setting the objective
Q = np.diag([100, 0])
R = np.diag([0])
lqr.set_objective(Q=Q, R=R)

# lqr setup
lqr.setup()

In [ ]:
# Define set point
xss = np.array([0.1, 0.0]).reshape(-1, 1)
lqr.set_setpoint(xss)

In [ ]:
env = create_mass_spring_damper_environment(max_steps=100)
initial_observation, _ = env.reset()

observation = initial_observation.copy()
observations = [observation]
actions = []

for _ in range(100):
    action = lqr.make_step(observation.reshape(-1, 1)).ravel()
    actions.append(action)
    observation, _, terminated, truncated, _ = env.step(action)
    observations.append(observation)
    if terminated or truncated:
        frames = env.render()
        break

observations = np.stack(observations)
actions = np.stack(actions)
env.close()

In [ ]:
media.show_video(frames, fps=1 / env.dt)

In [ ]:
fig, ax, graphics = do_mpc.graphics.default_plot(lqr.data)

## Exercise

- Change the cost matrices of the LQR problem above and try to find the best combinations. 
- Design an LQR controller for the inverted pendulum.

## Solution

### Inverted Pendulum

In [ ]:
# Initialize the controller
lqr = do_mpc.controller.LQR(inverted_pendulum_lin)

# Initialize the parameters
env = create_inverted_pendulum_environment()
lqr.settings.t_step = env.dt
lqr.settings.n_horizon = None  # infinite horizon

# Setting the objective
Q = np.diag([100, 0.1])
R = np.diag([0])
lqr.set_objective(Q=Q, R=R)

# lqr setup
lqr.setup()

In [ ]:
# Define set point
xss = np.array([0.0, 0.0]).reshape(-1, 1)
lqr.set_setpoint(xss)

In [ ]:
max_steps = 500
env = create_inverted_pendulum_environment(max_steps=max_steps)
initial_observation, _ = env.reset()

observation = initial_observation.copy()
observations = [observation]
actions = []

for _ in range(max_steps):
    action = lqr.make_step(observation[[1, 3]].reshape(-1, 1)).ravel()
    actions.append(action)
    observation, _, terminated, truncated, _ = env.step(action)
    observations.append(observation)
    if terminated or truncated:
        frames = env.render()
        break

observations = np.stack(observations)
actions = np.stack(actions)
env.close()

In [ ]:
media.show_video(frames, fps=1 / env.dt)

In [ ]:
fig, ax, graphics = do_mpc.graphics.default_plot(lqr.data, aux_list=[])

# Model Predictive Control

Unfortunately, the analytically convenient linear quadratic problem formulations are often not satisfactory. There are two main reasons for this:

- The system may be nonlinear, and it may be inappropriate to use for
  control purposes. Moreover, some of the control variables may be naturally
  discrete, and this is incompatible with the linear system viewpoint.
  
- There may be control and/or state constraints, which are not handled
  adequately through quadratic penalty terms in the cost function. For
  example, the motion of a car may be constrained by the presence of
  obstacles and hardware limitations.
  The solution obtained from a linear quadratic model may not be suitable for such
  a problem, because quadratic penalties treat constraints "softly"
  and may produce trajectories that violate the constraints.

A major difference between MPC and finite-state stochastic control problems
that are popular in the RL/artificial intelligence literature is that in MPC
the state and control spaces are continuous/infinite, such as for example
in self-driving cars, the control of aircraft and drones, or the operation of
chemical processes

## Iterative Linear Quadratic Regulator

## Differential Dynamic Programming

### Mass-Spring-Damper

In [ ]:
mpc_params = {
    "n_robust": 0,
    "n_horizon": 20,
    "t_step": env.dt,
    "store_full_solution": True,
}

xss = np.array([0.2, 0.0])
distance_cost = casadi.sumsqr(mass_spring_damper.x.cat - xss)
terminal_cost = distance_cost
stage_cost = distance_cost
input_penalty = 0.0

mpc = do_mpc.controller.MPC(mass_spring_damper)
mpc.set_param(**mpc_params)
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)
mpc.set_rterm(force=input_penalty)

In [ ]:
x_max = 2.0
u_max = 20

# lower and upper bounds of the states
mpc.bounds["lower", "_x", "position"] = -x_max
mpc.bounds["upper", "_x", "position"] = x_max

# lower and upper bounds of the input
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

In [ ]:
mpc.setup()

In [ ]:
mpc.set_initial_guess()

In [ ]:
%%capture
max_steps = 300
env = create_mass_spring_damper_environment(max_steps=max_steps)
observation, _ = env.reset()

for _ in range(max_steps):
    action = mpc.make_step(observation).ravel()
    observation, _, terminated, truncated, _ = env.step(action)
    if terminated or truncated:
        frames = env.render()
        break

env.close()

In [ ]:
media.show_video(frames, fps=1 / env.dt)

In [ ]:
fig, ax, graphics = do_mpc.graphics.default_plot(mpc.data)

In [ ]:
def update(t_ind):
    graphics.plot_results(t_ind)
    graphics.plot_predictions(t_ind)


frames_idx = [int(i * len(frames) / 50) for i in range(50)]
anim = FuncAnimation(fig, update, frames=frames_idx, repeat=False, interval=300)
HTML(anim.to_html5_video())

## Exercise

- Design an MPC controller for:
  - The linearized inverted pendulum.
  - The non-linear inverted pendulum.
- For each case, try different cost functions:
  - $\sum \theta^2$
  - $\sum |\theta|$
  - $E_{\text{kinetic}} - E_{\text{potential}}$

## Solution

### Linear Inverted Pendulum

In [ ]:
env = create_inverted_pendulum_environment()

energy_cost = inverted_pendulum_lin.aux["E_kinetic"] - inverted_pendulum_lin.aux["E_potential"]
xss = np.array([0.0, 0.0])
# distance_cost = casadi.sumsqr(inverted_pendulum_lin.x.cat - xss)
distance_cost = casadi.norm_1(inverted_pendulum_lin.x.cat - xss)
terminal_cost = distance_cost
stage_cost = distance_cost
input_penalty = 1e-4

mpc_params = {
    "n_robust": 0,
    "n_horizon": 20,
    "t_step": env.dt,
    "store_full_solution": True,
}
mpc = do_mpc.controller.MPC(inverted_pendulum_lin)
mpc.set_param(**mpc_params)
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)
mpc.set_rterm(force=input_penalty)

In [ ]:
u_max = 3

# lower and upper bounds of the input
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

In [ ]:
mpc.setup()

In [ ]:
mpc.x0 = np.zeros(2)
mpc.set_initial_guess()

In [ ]:
%%capture
max_steps = 300
env = create_inverted_pendulum_environment(max_steps=max_steps)
observation, _ = env.reset()

for _ in range(max_steps):
    action = mpc.make_step(observation[[1, 3]]).ravel()
    observation, _, terminated, truncated, _ = env.step(action)
    if terminated or truncated:
        frames = env.render()
        break

env.close()

In [ ]:
media.show_video(frames, fps=1 / env.dt)

In [ ]:
fig, ax, graphics = do_mpc.graphics.default_plot(mpc.data)

In [ ]:
def update(t_ind):
    graphics.plot_results(t_ind)
    graphics.plot_predictions(t_ind)


frames_idx = [int(i * len(frames) / 50) for i in range(50)]
anim = FuncAnimation(fig, update, frames=frames_idx, repeat=False, interval=500)
HTML(anim.to_html5_video())

### Non-Linear Inverted Pendulum

In [ ]:
env = create_inverted_pendulum_environment()

energy_cost = inverted_pendulum.aux["E_kinetic"] - inverted_pendulum.aux["E_potential"]
distance_cost = casadi.bilin(np.diag([1, 1000, 0, 0]), inverted_pendulum.x.cat)
terminal_cost = distance_cost
stage_cost = distance_cost
input_penalty = 0

mpc_params = {
    "n_robust": 0,
    "n_horizon": 30,
    "t_step": env.dt,
    "store_full_solution": True,
}
mpc = do_mpc.controller.MPC(inverted_pendulum)
mpc.set_param(**mpc_params)
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)
mpc.set_rterm(force=input_penalty)

In [ ]:
x_max = 1.0
mpc.bounds["lower", "_x", "position"] = -x_max
mpc.bounds["upper", "_x", "position"] = x_max

# lower and upper bounds of the input
u_max = 3
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

In [ ]:
mpc.setup()

In [ ]:
mpc.x0 = np.zeros(4)
mpc.set_initial_guess()

In [ ]:
%%capture
max_steps = 300
env = create_inverted_pendulum_environment(max_steps=max_steps)
observation, _ = env.reset()

for _ in range(max_steps):
    action = mpc.make_step(observation).ravel()
    observation, _, terminated, truncated, _ = env.step(action)
    if terminated or truncated:
        frames = env.render()
        break

env.close()

In [ ]:
media.show_video(frames, fps=1 / env.dt)

In [ ]:
fig, ax, graphics = do_mpc.graphics.default_plot(mpc.data)

In [ ]:
def update(t_ind):
    graphics.plot_results(t_ind)
    graphics.plot_predictions(t_ind)


frames_idx = [int(i * len(frames) / 30) for i in range(30)]
anim = FuncAnimation(fig, update, frames=frames_idx, repeat=False, interval=500)
HTML(anim.to_html5_video())

# Summary

<img src="_static/images/aai-institute-cover.svg" alt="Snow" style="width:100%;">
<div class="md-slide title">Thank you for the attention!</div>

# References

- [<b id="slotine_applied_1991">[Slotine 1991]</b>](#slotine_applied_1991-back) [Applied nonlinear control.](https://www.academia.edu/download/33582713/Applied_Nonlinear_Control_Slotine.pdf) Slotine, Jean-Jacques E., and Weiping Li. Vol. 199, no. 1. Englewood Cliffs, NJ: Prentice hall. 1991.

- [<b id="grune_nonlinear_2017">[Lars Nonlinear 2017]</b>](#grune_nonlinear_2017-back) [Nonlinear model predictive control.](https://link.springer.com/chapter/10.1007/978-3-319-46024-6_3) Grüne, Lars, Jürgen Pannek, Lars Grüne, and Jürgen Pannek. Springer International Publishing, 2017.

- [<b id="bertsekas_lessons_2022">[Bertsektas, Dimitri P, 2022]</b>](#bertsekas_lessons_2022-back) [Lessons from AlphaZero for Optimal, Model Predictive, and Adaptive Control](http://web.mit.edu/dimitrib/www/LessonsfromAlphazero.pdf) - Dimitri P. Bertsektas. 2022.
- [<b id="spencer_optimal_2023">[Spencer et al. 2023]</b>](#spencer_optimal_2023-back) [AA 203: Optimal and Learning-Based Control.](https://stanfordasl.github.io/aa203/sp2223/) Optimal control solution techniques for systems with known and unknown dynamics. 2023.